# Data Extract and upload for NADAbase

## Steps

1. Extract Episodes for the period from Communicare
2. Extract ATOM data
3. Match Assessments with Episodes
4. Generate the upload Survey.txt file.

In [1]:
# import os
import pandas as pd
# import numpy as np
import mylogger
# from nada_load import load_and_parse_csv
from data_config import EstablishmentID_Program
from utils.df_xtrct_prep import extract_prep_atom_data #load_and_parse_episode_csvs

logger = mylogger.get(__name__)
# # List of column names in the CSV
# column_names = ['ESTABLISHMENT IDENTIFIER', 'GEOGRAPHICAL LOCATION', 'PMSEpisodeID', 'PMSPersonID', 'DOB', 'DOB STATUS', 'SEX', 'COUNTRY OF BIRTH', 'INDIGENOUS STATUS', 'PREFERRED LANGUAGE', 'SOURCE OF INCOME', 'LIVING ARRANGEMENT', 'USUAL ACCOMMODATION', 'CLIENT TYPE', 'PRINCIPAL DRUG OF CONCERN', 'PDCSubstanceOfConcern', 'ILLICIT USE', 'METHOD OF USE PRINCIPAL DRUG', 'INJECTING DRUG USE', 'SETTING', 'CommencementDate', 'POSTCODE', 'SOURCE OF REFERRAL', 'MAIN SERVICE', 'EndDate', 'END REASON', 'REFERRAL TO ANOTHER SERVICE', 'FAMILY NAME', 'GIVEN NAME', 'MIDDLE NAME', 'TITLE', 'SLK', 'MEDICARE NUMBER', 'PROPERTY NAME', 'UNIT FLAT NUMBER', 'STREET NUMBER', 'STREET NAME', 'SUBURB']




In [2]:
# Global variables
ep_source_filename = 'DATS_AllPrograms_01072023-30122023' #'DATS_Annual_AllPrograms_FY22-23'
ep_datasource_file_path = f"./data/in/NSW_CSV/{ep_source_filename}.csv"

extract_start_date = 20130501
extract_end_date = 20231231

fname = f"{extract_start_date}_{extract_end_date}" # TODO :forNaada

active_clients_start_date ='2023-07-01' 
active_clients_end_date = '2023-12-31'

results_folder = "./data/out/"

In [3]:
from datetime import datetime

def custom_date_parser(date_str):
     # Check if the input is NaN (Not a Number)
    if pd.isna(date_str):
        return datetime.now().date()  # 
    
    # if not isinstance(date_str, int) :
    #     return datetime.now().date()  # Replace with today's date
    # Check if the date_str is not a string or is an empty string
    date_str = str(int(date_str)).zfill(8)
    
    # # Check if the string length is less than 8 characters
    # if len(date_str) < 8:
    #     # Possibly handle or log this case, as it's an unexpected format
    #     return None  # or choose an appropriate default value

    try:
        return pd.to_datetime(date_str, format='%d%m%Y').date()
    except ValueError:
        # Handle the case where the date_str is not a valid date
        # You can log this error if needed
        return None  # or choose an appropriate default value


In [8]:
# # List of columns we care about
columns_of_interest = ['ESTABLISHMENT IDENTIFIER', 'GEOGRAPHICAL LOCATION', 'EPISODE ID','PERSON ID', 'SPECIFY DRUG OF CONCERN', 'PRINCIPAL DRUG OF CONCERN', 'START DATE', 'END DATE', 'SLK']
rename_columns = {
    'SPECIFY DRUG OF CONCERN': 'PDCSubstanceOfConcern',
    'START DATE': 'CommencementDate',
    'END DATE': 'EndDate',
    'EPISODE ID': 'PMSEpisodeID',
    'PERSON ID': 'PMSPersonID',
    'PRINCIPAL DRUG OF CONCERN': 'PDCCode'
}

        # return pd.to_datetime(date,  dayfirst=True)

# from utils.df_xtrct_prep
#Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.  
def load_and_parse_csv(filepath):
    # Load the CSV

    df = pd.read_csv(filepath,  usecols=columns_of_interest)
                    # , date_format='%d%m%Y', parse_dates=['START DATE', 'END DATE'])
    # Apply the convert_date function to the date columns
    df['START DATE'] = df['START DATE'].apply(custom_date_parser)
    df['END DATE'] = df['END DATE'].apply(custom_date_parser)    
    df.rename(columns=rename_columns, inplace=True)
   

    # df['CommencementDate'] = pd.to_datetime(df['CommencementDate'], format='%d%m%Y')
    # df['EndDate'] = pd.to_datetime(df['EndDate'], format='%d%m%Y')    
    return df


In [9]:

# ep_df  = load_and_parse_episode_csvs(ep_datasource_file_path)
ep_df = load_and_parse_csv(ep_datasource_file_path)
ep_df['Program'] = ep_df['ESTABLISHMENT IDENTIFIER'].map(EstablishmentID_Program)

In [10]:
ep_df.head(10)

,SLK,CommencementDate,EndDate,PMSEpisodeID,PMSPersonID,PDCSubstanceOfConcern,PDCCode,ESTABLISHMENT IDENTIFIER,GEOGRAPHICAL LOCATION,Program
0,USEUK190519821,2021-03-19,2024-01-05,1852,170,Alcohol,2101,12QQ03076,10550,SAPPHIRE
1,AMOAT241019791,2021-12-16,2024-01-05,23373,143,Alcohol,2101,12QQ03076,10550,SAPPHIRE
2,OUDES271119791,2022-04-12,2024-01-05,39191,100,Methamphetamine,3103,12QQ03076,10550,SAPPHIRE
3,UCELE131119691,2022-08-03,2024-01-05,48350,5324,Alcohol,2101,12QQ03076,10550,SAPPHIRE
4,RO2AC191120001,2022-09-08,2023-11-28,51655,8876,Alcohol,2101,12QQ03076,10550,SAPPHIRE
5,RAHAR221019911,2022-09-06,2023-12-20,51318,7331,Alcohol,2101,12QQ03076,10550,SAPPHIRE
6,OL2IR141219712,2022-09-15,2024-01-05,53719,139,Alcohol,2101,12QQ03076,10550,SAPPHIRE
7,AYULA290319921,2022-10-06,2024-01-05,54450,5526,Alcohol,2101,12QQ03076,10550,SAPPHIRE
8,MIHAR301119831,2022-11-25,2024-01-05,59206,5158,Alcohol,2101,12QQ03076,10550,SAPPHIRE
9,URIYL100819842,2022-10-11,2024-01-05,61651,242,Alcohol,2101,12QQ03076,10550,SAPPHIRE


In [11]:
len(ep_df), min(ep_df.CommencementDate), max(ep_df.CommencementDate)

(787, datetime.date(2016, 11, 11), datetime.date(2023, 12, 22))

In [12]:
ep_df['Program'] = ep_df['ESTABLISHMENT IDENTIFIER'].map(EstablishmentID_Program)
ep_df.drop(columns=['ESTABLISHMENT IDENTIFIER'], inplace=True)

In [13]:
ep_df.columns

Index(['SLK', 'CommencementDate', 'EndDate', 'PMSEpisodeID', 'PMSPersonID',
       'PDCSubstanceOfConcern', 'PDCCode', 'GEOGRAPHICAL LOCATION', 'Program'],
      dtype='object')

In [14]:
from utils.environment import MyEnvironmentConfig

In [15]:
# to adjust the ODC parser
extract_start_date = 20230701
extract_end_date = 20231231

fname = f"{extract_start_date}-{extract_end_date}" # TODO :forNaada
MyEnvironmentConfig().setup('prod')

processed_df = extract_prep_atom_data(extract_start_date, extract_end_date
                                 , active_clients_start_date
                                 , active_clients_end_date
                                 , fname, purpose='NADA')

In [16]:
# processed_df = ['SLK',	'RowKey','ClientType',	'AssessmentDate', PDCSubstanceOrGambling	PDCMethodOfUse	PDCDaysInLast28	PDCUnits	PDCHowMuchPerOccasion	PDCAgeLastUsed	PDCGoals	PDC]
processed_df

,SLK,RowKey,Program,Staff,SurveyName,AssessmentDate,CCareID,ClientType,IARDST,IndigenousStatus,...,Possibilities2,WhatCouldGetInTheWay2,PDCSubstanceOrGambling,PDCMethodOfUse,PDCDaysInLast28,PDCUnits,PDCHowMuchPerOccasion,PDCAgeLastUsed,PDCGoals,PDC
1047,RITAA010719962,MURMPP_ITSP_20230703,MURMPP,Mikaela.Feltham,ATOM ITSP Review Assessment,2023-07-03,NaN,ownuse,NaN,NaN,...,NaN,NaN,Ethanol,Ingest,0.0,standard drinks,0.0,26.0,Maintain Abstinence,Ethanol
418,ES2ES111119821,MONPATH_INAS_20230703,MONPATH,Helen.Waite,ATOM Initial Assessment,2023-07-03,NaN,ownuse,NaN,Torres Strait Islander but not Aboriginal,...,NaN,NaN,Ethanol,Ingest,28.0,standard drinks,9.0,16.0,Cease Use,Ethanol
442,EWNIM060119762,EUROPATH_INAS_20230703,EUROPATH,Paul.Crowe,ATOM Initial Assessment,2023-07-03,NaN,ownuse,NaN,Neither Aboriginal nor Torres Strait Islander,...,NaN,NaN,Ethanol,Ingest,11.0,standard drinks,0.0,47.0,Maintain Abstinence,Ethanol
1075,SACAR270619511,MURMICE_ITSP_20230703,MURMICE,Tina.Raccanello,ATOM ITSP Review Assessment,2023-07-03,NaN,ownuse,NaN,NaN,...,NaN,NaN,Methamphetamine,Ingest,15.0,points,1.0,71.0,Reduce Use,Methamphetamine
64,ALHEN140319901,COCO_SITK_20230703,COCO,Mysh.Kirkwood,Arcadia House Assessments,2023-07-03,NaN,ownuse,NaN,NaN,...,NaN,NaN,Ethanol,Ingest,28.0,standard drinks,24.5,33.0,Cease Use,Ethanol
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,ILION150419561,TSS_ITSP_20231222,TSS,Emiltus.Emmanuel,ATOM ITSP Review Assessment,2023-12-22,NaN,ownuse,NaN,NaN,...,NaN,NaN,Methamphetamine,Inject,19.0,points,1.0,NaN,Maintain Current level of use,Methamphetamine
13,ADIRO150619781,GOLBICE_ITSP_20231222,GOLBICE,James.Costello,ATOM ITSP Review Assessment,2023-12-22,NaN,ownuse,NaN,NaN,...,NaN,NaN,"Cannabinoids and Related Drugs, n.f.d.",Smoke,28.0,grams,1.0,45.0,Cease Use,"Cannabinoids and Related Drugs, n.f.d."
1093,TEARO220820081,GOLBGNRL_ITSP_20231222,GOLBGNRL,James.Costello,ATOM ITSP Review Assessment,2023-12-22,NaN,ownuse,NaN,NaN,...,NaN,NaN,"Cannabinoids and Related Drugs, n.f.d.",Smoke,4.0,grams,0.5,14.0,Reduce Harms,"Cannabinoids and Related Drugs, n.f.d."
761,ODEMI271219932,SAPPHIRE_INAS_20231222,SAPPHIRE,Kylie.Wood,ATOM Initial Assessment,2023-12-22,NaN,ownuse,NaN,Neither Aboriginal nor Torres Strait Islander,...,NaN,NaN,Methamphetamine,Inhale (vapour),20.0,points,3.0,28.0,Reduce Use,Methamphetamine


In [83]:
# from utils.io import write_parquet
# processed_filepath = f"./data/processed/atom_NADA_{fname}.parquet"
# write_parquet(processed_df, processed_filepath) # don't force overwrite
# # ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column K10Q12 with type object')

ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column K10Q12 with type object')

In [17]:
atom_df  = processed_df.copy()
atom_df

,SLK,RowKey,Program,Staff,SurveyName,AssessmentDate,CCareID,ClientType,IARDST,IndigenousStatus,...,Possibilities2,WhatCouldGetInTheWay2,PDCSubstanceOrGambling,PDCMethodOfUse,PDCDaysInLast28,PDCUnits,PDCHowMuchPerOccasion,PDCAgeLastUsed,PDCGoals,PDC
1047,RITAA010719962,MURMPP_ITSP_20230703,MURMPP,Mikaela.Feltham,ATOM ITSP Review Assessment,2023-07-03,NaN,ownuse,NaN,NaN,...,NaN,NaN,Ethanol,Ingest,0.0,standard drinks,0.0,26.0,Maintain Abstinence,Ethanol
418,ES2ES111119821,MONPATH_INAS_20230703,MONPATH,Helen.Waite,ATOM Initial Assessment,2023-07-03,NaN,ownuse,NaN,Torres Strait Islander but not Aboriginal,...,NaN,NaN,Ethanol,Ingest,28.0,standard drinks,9.0,16.0,Cease Use,Ethanol
442,EWNIM060119762,EUROPATH_INAS_20230703,EUROPATH,Paul.Crowe,ATOM Initial Assessment,2023-07-03,NaN,ownuse,NaN,Neither Aboriginal nor Torres Strait Islander,...,NaN,NaN,Ethanol,Ingest,11.0,standard drinks,0.0,47.0,Maintain Abstinence,Ethanol
1075,SACAR270619511,MURMICE_ITSP_20230703,MURMICE,Tina.Raccanello,ATOM ITSP Review Assessment,2023-07-03,NaN,ownuse,NaN,NaN,...,NaN,NaN,Methamphetamine,Ingest,15.0,points,1.0,71.0,Reduce Use,Methamphetamine
64,ALHEN140319901,COCO_SITK_20230703,COCO,Mysh.Kirkwood,Arcadia House Assessments,2023-07-03,NaN,ownuse,NaN,NaN,...,NaN,NaN,Ethanol,Ingest,28.0,standard drinks,24.5,33.0,Cease Use,Ethanol
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,ILION150419561,TSS_ITSP_20231222,TSS,Emiltus.Emmanuel,ATOM ITSP Review Assessment,2023-12-22,NaN,ownuse,NaN,NaN,...,NaN,NaN,Methamphetamine,Inject,19.0,points,1.0,NaN,Maintain Current level of use,Methamphetamine
13,ADIRO150619781,GOLBICE_ITSP_20231222,GOLBICE,James.Costello,ATOM ITSP Review Assessment,2023-12-22,NaN,ownuse,NaN,NaN,...,NaN,NaN,"Cannabinoids and Related Drugs, n.f.d.",Smoke,28.0,grams,1.0,45.0,Cease Use,"Cannabinoids and Related Drugs, n.f.d."
1093,TEARO220820081,GOLBGNRL_ITSP_20231222,GOLBGNRL,James.Costello,ATOM ITSP Review Assessment,2023-12-22,NaN,ownuse,NaN,NaN,...,NaN,NaN,"Cannabinoids and Related Drugs, n.f.d.",Smoke,4.0,grams,0.5,14.0,Reduce Harms,"Cannabinoids and Related Drugs, n.f.d."
761,ODEMI271219932,SAPPHIRE_INAS_20231222,SAPPHIRE,Kylie.Wood,ATOM Initial Assessment,2023-12-22,NaN,ownuse,NaN,Neither Aboriginal nor Torres Strait Islander,...,NaN,NaN,Methamphetamine,Inhale (vapour),20.0,points,3.0,28.0,Reduce Use,Methamphetamine


In [18]:
atom_df = atom_df.loc[ (atom_df.AssessmentDate>= active_clients_start_date ) & (atom_df.AssessmentDate <= active_clients_end_date)]

In [19]:
# Ensure that AssessmentDate is in datetime format
atom_df['AssessmentDate'] = pd.to_datetime(atom_df['AssessmentDate'], errors='coerce')

In [20]:
# atom_df.drop(atom_df[atom_df.Program.isin(['TSS', 'ARCA', 'COCO', 'PSYNSW' ])].index, inplace=True)
non_nsw_programs = ['TSS', 'ARCA', 'COCO', 'PSYNSW' ]
atom_df = atom_df[~atom_df['Program'].isin(non_nsw_programs)] 


In [21]:
len(ep_df), len(atom_df) 

(787, 763)

In [22]:
atom_df.Program.value_counts()

Program
SAPPHIRE     155
EUROPATH     126
MURMICE      125
GOLBICE      113
GOLBGNRL      59
MONPATH       55
MURMWIO       51
MURMPP        33
ALONGSIDE     23
BEGAPATH      16
BegaMHHub      7
TSS            0
COCO           0
ARCA           0
Name: count, dtype: int64

In [27]:
from match_audit import match_assessments

In [30]:
# def match_assessments(episodes_df, atoms_df): #pdc_substance_mapping
#     # Apply the mapping to the ESTABLISHMENT IDENTIFIER and PDCSubstanceOfConcern columns in episodes_df
#     # episodes_df['ESTABLISHMENT IDENTIFIER'] = episodes_df['ESTABLISHMENT IDENTIFIER'].map(establishment_program_mapping)
#     # episodes_df['PDCSubstanceOfConcern'] = episodes_df['PDCSubstanceOfConcern'].map(pdc_substance_mapping)

#     # Merge the dataframes on SLK and Program
#     merged_df = pd.merge(episodes_df, atoms_df, how='inner', left_on=['SLK', 'Program'], right_on=['SLK', 'Program'])

#     # Filter rows where AssessmentDate falls within CommencementDate and EndDate (or after CommencementDate if EndDate is NaN)
#     matched_df = merged_df.loc[((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) & 
#                                 (merged_df['AssessmentDate'] <= merged_df['EndDate'])) |
#                                ((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) & 
#                                 (merged_df['EndDate'].isna()))]

#     # Check if PDCSubstanceOfConcern matches
#     # mismatched_df = matched_df.loc[matched_df['PDCSubstanceOfConcern_x'] != matched_df['PDCSubstanceOfConcern_y']]

#     # if len(mismatched_df) > 0:
#     #     logger.info(f"There are {len(mismatched_df)} rows where PDCSubstanceOfConcern does not match.")
#     #     logger.info(mismatched_df)

#     return matched_df

In [29]:
def get_mask_datefit(row, slack_days=30):
    # Convert to datetime if not already in that format
    # assessment_date = pd.to_datetime(row['AssessmentDate'], errors='coerce')
    # commencement_date = pd.to_datetime(row['CommencementDate'], errors='coerce')
    # end_date = pd.to_datetime(row['EndDate'], errors='coerce')

    # Create a Timedelta for slack days
    slack_td = pd.Timedelta(days=slack_days)

    # Check conditions
    after_commencement = row['AssessmentDate'].date() >= (row['CommencementDate'] - slack_td)
    before_end_date = row['AssessmentDate'].date() <= (row['EndDate'] + slack_td)

    return after_commencement and before_end_date


def match_assessments(episodes_df, atoms_df, matching_ndays_slack: int):

    # Merge the dataframes on SLK and Program
    df = pd.merge(episodes_df, atoms_df, how='inner', left_on=[
                  'SLK', 'Program'], right_on=['SLK', 'Program'])

    # Filter rows where AssessmentDate falls within CommencementDate and EndDate (or after CommencementDate if EndDate is NaN)
    mask = df.apply(get_mask_datefit, slack_days=matching_ndays_slack, axis=1)
    # mask = get_mask_datefit(df['AssessmentDate'], df['CommencementDate'],
    #                         df['EndDate'], slack_days=matching_ndays_slack)
    filtered_df = df[mask]
    # matched_df = merged_df.loc[((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) &
    #                             (merged_df['AssessmentDate'] <= merged_df['EndDate'])) |
    #                            ((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) &
    #                             (merged_df['EndDate'].isna()))]

    # Check if PDCSubstanceOfConcern matches
    # mismatched_df = matched_df.loc[matched_df['PDCSubstanceOfConcern_x'] != matched_df['PDCSubstanceOfConcern_y']]

    # if len(mismatched_df) > 0:
    #     logger.info(f"There are {len(mismatched_df)} rows where PDCSubstanceOfConcern does not match.")
    #     logger.info(mismatched_df)

    return filtered_df

In [30]:
matching_ndays_slack = 14
matched_df = match_assessments(ep_df, atom_df, matching_ndays_slack)

In [34]:
# matched_df[['SLK', 'CommencementDate', 'AssessmentDate', 'EndDate']].head(30)
matched_df

,SLK,CommencementDate,EndDate,PMSEpisodeID,PMSPersonID,PDCSubstanceOfConcern,PDCCode,GEOGRAPHICAL LOCATION,Program,RowKey,...,Possibilities2,WhatCouldGetInTheWay2,PDCSubstanceOrGambling,PDCMethodOfUse,PDCDaysInLast28,PDCUnits,PDCHowMuchPerOccasion,PDCAgeLastUsed,PDCGoals,PDC
0,USEUK190519821,2021-03-19,2024-01-05,1852,170,Alcohol,2101,10550,SAPPHIRE,SAPPHIRE_ITSP_20230801,...,NaN,NaN,"Cannabinoids and Related Drugs, n.f.d.",Smoke,28.0,grams,2.0,39.0,Reduce Use,"Cannabinoids and Related Drugs, n.f.d."
1,RO2AC191120001,2022-09-08,2023-11-28,51655,8876,Alcohol,2101,10550,SAPPHIRE,SAPPHIRE_ITSP_20230809,...,NaN,NaN,Ethanol,Ingest,3.0,standard drinks,2.0,21.0,Cease Use,Ethanol
2,RO2AC191120001,2022-09-08,2023-11-28,51655,8876,Alcohol,2101,10550,SAPPHIRE,SAPPHIRE_ITSP_20230814,...,NaN,NaN,Ethanol,Ingest,3.0,standard drinks,2.0,21.0,Cease Use,Ethanol
3,RO2AC191120001,2022-09-08,2023-11-28,51655,8876,Alcohol,2101,10550,SAPPHIRE,SAPPHIRE_ITSP_20231011,...,NaN,NaN,Ethanol,Ingest,3.0,standard drinks,2.0,21.0,Cease Use,Ethanol
4,RAHAR221019911,2022-09-06,2023-12-20,51318,7331,Alcohol,2101,10550,SAPPHIRE,SAPPHIRE_INAS_20230809,...,NaN,NaN,Ethanol,Ingest,0.0,standard drinks,17.0,31.0,Maintain Abstinence,Ethanol
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,OLYES140519952,2023-08-21,2024-01-05,84006,9752,Alcohol,2101,17751,MURMPP,MURMPP_INAS_20230821,...,NaN,NaN,Ethanol,Ingest,12.0,standard drinks,12.0,28.0,Reduce Use,Ethanol
739,HOPOB040719811,2023-10-03,2023-12-13,89462,9893,Methamphetamine,3103,17751,MURMPP,MURMPP_INAS_20231003,...,NaN,NaN,Methamphetamine,Inject,0.0,points,10.0,42.0,Maintain Abstinence,Methamphetamine
740,RAFRA030519901,2023-11-07,2024-01-05,93785,9322,Methamphetamine,3103,17751,MURMPP,MURMPP_INAS_20231107,...,NaN,NaN,Ethanol,Ingest,0.0,standard drinks,2.0,32.0,Maintain Abstinence,Ethanol
741,AR2EL091019852,2023-12-07,2024-01-05,97444,2183,Cannabinoids,7101,17751,MURMPP,MURMPP_INAS_20231207,...,NaN,NaN,"Cannabinoids and Related Drugs, n.f.d.",Smoke,28.0,grams,2.0,38.0,Reduce Harms,"Cannabinoids and Related Drugs, n.f.d."


In [39]:
# PDCSubstanceOrGambling atom_df.columns
# ep_df.columns # PDCSubstanceOfConcern # PDCCode <- from episode

# matched_df.PDC <- from ATOM
# matched_df.PDCSubstanceOfConcern <- from Episode

0              Alcohol
1              Alcohol
2              Alcohol
3              Alcohol
4              Alcohol
            ...       
738            Alcohol
739    Methamphetamine
740    Methamphetamine
741       Cannabinoids
742    Methamphetamine
Name: PDCSubstanceOfConcern, Length: 690, dtype: object

In [22]:
# list(matched_df.PDC.unique())
# list(ep_df.PDCSubstanceOfConcern.unique())
ep_df.PDCSubstanceOfConcern.value_counts()

PDCSubstanceOfConcern
Alcohol                                560
Methamphetamine                        425
Cannabinoids                           199
Cannabinoids and Related Drugs, nec     19
Amphetamine                             12
Heroin                                  11
Cocaine                                 11
Nicotine/tobacco                         7
Benzodiazepines, nec                     4
Cannabinoid agonists                     3
Opioid Antagonists, nec                  2
Oxycodone                                2
MDMA/Ecstasy                             2
Amphetamines, nec                        2
Methadone                                2
Alcohols, nec                            2
Nitrous oxide                            1
Acetylsalicylic acid                     1
Diazepam                                 1
Alprazolam                               1
Methcathinone                            1
LSD                                      1
Name: count, dtype: int64

In [46]:
import numpy as np

conditions = [
    # matched_df['PDCSubstanceOfConcern'].isin(['Ethanol', 'Alcohols, n.e.c.']),
     matched_df['PDCSubstanceOfConcern'].isin(['Alcohol']),
    matched_df['PDCCode'].between(7100, 7199),
    matched_df['PDCCode'].between(3100, 3199),
    matched_df['PDCSubstanceOfConcern'].isin(['Pharmaceutical Opioids, n.f.d.']),
    matched_df['PDCCode'].between(2400, 2499),
    matched_df['PDCCode'] == 1202,
    matched_df['PDCCode'].between(1100, 1399),
    ~matched_df['PDCSubstanceOfConcern'].isin(['Nicotine', 'Cocaine'])
]
# 'Alcohol', 'Cannabinoids and Related Drugs, nec', 'Heroin',
#        'Cannabinoids', 'Benzodiazepines, nec', 'Methamphetamine',
#        'Nicotine/tobacco', 'Cocaine', nan, 'Dexamphetamine',
#        'Amphetamine', 'Other Volatile Solvents, nec', 'MDMA/Ecstasy',
#        'Gambling', 'Alprazolam', 'Cannabinoid agonists', 'Buprenorphine'

choices = [
    'Alcohol',
    'Cannabis',
    'Amphetamines',
    'Other Opioid Use',
    'Tranquilisers',
    'Heroin',
    'Other Opioid Use',
    'Another drug use'
]

matched_df['PDCFinal'] = np.select(conditions, choices, default=matched_df['PDCSubstanceOfConcern'])

In [48]:
# Since the primary key in episodes_df is PMSEpisodeID, and in atoms_df it is a combination of SLK and Program, 
# you can use the groupby method on the merged_df to group by SLK and Program, 
# and then check if there is more than one unique PMSEpisodeID for any group.

# grouped = matched_df.groupby(['SLK', 'Program'])['PMSEpisodeID'].nunique()
grouped = matched_df.groupby(['SLK', 'RowKey'])['PMSEpisodeID'].nunique()
duplicates = grouped[grouped > 1]


In [49]:
duplicates

SLK             RowKey                
AHYAT141219641  SAPPHIRE_ITSP_20231128    2
ANAND160719921  GOLBICE_INAS_20230927     2
HALOU141019932  SAPPHIRE_INAS_20231205    2
OUGIR061119822  MONPATH_INAS_20231026     2
Name: PMSEpisodeID, dtype: int64

In [50]:
# SLK + Program
onedupe = matched_df.loc[(matched_df.SLK =='AHYAT141219641') & (matched_df.Program == 'SAPPHIRE')]

In [55]:
onedupe [['Program', 'SLK', 'CommencementDate', 'AssessmentDate', 'EndDate', 'PMSEpisodeID', 'RowKey']]

,Program,SLK,CommencementDate,AssessmentDate,EndDate,PMSEpisodeID,RowKey
62,SAPPHIRE,AHYAT141219641,2023-07-11,2023-07-11,2023-11-28,79760,SAPPHIRE_INAS_20230711
63,SAPPHIRE,AHYAT141219641,2023-07-11,2023-09-19,2023-11-28,79760,SAPPHIRE_ITSP_20230919
64,SAPPHIRE,AHYAT141219641,2023-07-11,2023-11-28,2023-11-28,79760,SAPPHIRE_ITSP_20231128
67,SAPPHIRE,AHYAT141219641,2023-11-28,2023-11-28,2024-01-05,99200,SAPPHIRE_ITSP_20231128


In [61]:

# Get the keys for the duplicate rows
duplicate_keys = duplicates.index

# Filter matched_df to show only rows that match the duplicate keys
duplicate_rows_df = matched_df[matched_df.set_index(['SLK', 'RowKey']).index.isin(duplicate_keys)]

# Show all rows in the resulting DataFrame
pd.set_option('display.max_rows', None)  # Set this to display all rows



                SLK CommencementDate     EndDate  PMSEpisodeID  PMSPersonID   
64   AHYAT141219641       2023-07-11  2023-11-28         79760         7021  \
67   AHYAT141219641       2023-11-28  2024-01-05         99200         7021   
97   HALOU141019932       2023-08-15  2023-11-22         83405         6649   
99   HALOU141019932       2023-11-22  2024-01-05         95563         6649   
211  OUGIR061119822       2023-10-18  2023-10-23         91030         6691   
213  OUGIR061119822       2023-10-23  2024-01-05         91631         6691   
496  ANAND160719921       2023-08-23  2023-09-13         84330         9773   
497  ANAND160719921       2023-09-27  2023-11-09         89234         9773   

    PDCSubstanceOfConcern  PDCCode  GEOGRAPHICAL LOCATION   Program   
64                Alcohol     2101                  10550  SAPPHIRE  \
67                Alcohol     2101                  10550  SAPPHIRE   
97           Cannabinoids     7101                  10550  SAPPHIRE   
99  

In [64]:
duplicate_rows_df[['SLK','RowKey',  'Program'  ,  'CommencementDate',   'EndDate' , 'PMSEpisodeID', 'AssessmentDate' ]]

,SLK,RowKey,Program,CommencementDate,EndDate,PMSEpisodeID,AssessmentDate
64,AHYAT141219641,SAPPHIRE_ITSP_20231128,SAPPHIRE,2023-07-11,2023-11-28,79760,2023-11-28
67,AHYAT141219641,SAPPHIRE_ITSP_20231128,SAPPHIRE,2023-11-28,2024-01-05,99200,2023-11-28
97,HALOU141019932,SAPPHIRE_INAS_20231205,SAPPHIRE,2023-08-15,2023-11-22,83405,2023-12-05
99,HALOU141019932,SAPPHIRE_INAS_20231205,SAPPHIRE,2023-11-22,2024-01-05,95563,2023-12-05
211,OUGIR061119822,MONPATH_INAS_20231026,MONPATH,2023-10-18,2023-10-23,91030,2023-10-26
213,OUGIR061119822,MONPATH_INAS_20231026,MONPATH,2023-10-23,2024-01-05,91631,2023-10-26
496,ANAND160719921,GOLBICE_INAS_20230927,GOLBICE,2023-08-23,2023-09-13,84330,2023-09-27
497,ANAND160719921,GOLBICE_INAS_20230927,GOLBICE,2023-09-27,2023-11-09,89234,2023-09-27


In [38]:
matched_df['Program'].value_counts()

Program
SAPPHIRE    17
MURMICE     15
EUROPATH    12
GOLBICE      9
BEGAPATH     4
MURMPP       4
MURMWIO      3
MONPATH      2
Name: count, dtype: int64

In [43]:
# matched_df.to_csv(f"{results_folder}matched_df.csv", index=False)

In [39]:
matched_df.PMSEpisodeID.unique().shape

(62,)

In [45]:
atom_df[ (atom_df.AssessmentDate >= active_clients_start_date ) & (atom_df.AssessmentDate <= active_clients_end_date)].count()

SLK                      1576
RowKey                   1576
Program                  1576
SurveyName               1576
AssessmentDate           1576
                         ... 
PDCMethodOfUse           1576
PDCHowMuchPerOccasion    1507
PDCUnits                 1495
PDCAgeLastUsed           1534
PDCGoals                 1569
Length: 186, dtype: int64

In [46]:
len(atom_df[ (atom_df.AssessmentDate >= active_clients_start_date ) & (atom_df.AssessmentDate <= active_clients_end_date)])

1576

In [66]:
len(matched_df.PMSEpisodeID.unique())

497

In [40]:
len(ep_df)


1302

In [43]:
matched_df

,GEOGRAPHICAL LOCATION,PMSEpisodeID,PMSPersonID,PDCSubstanceOfConcern,CommencementDate,EndDate,SLK,Program,AssessmentDate,Staff,RowKey,PDC
1,10550,51655,8876,Alcohol,2022-09-08,NaT,RO2AC191120001,SAPPHIRE,2023-10-11,Kylie.Wood,SAPPHIRE_ITSP_20231011,Ethanol
2,10550,51318,7331,Alcohol,2022-09-06,NaT,RAHAR221019911,SAPPHIRE,2023-10-25,Kylie.Wood,SAPPHIRE_ITSP_20231025,Ethanol
3,10550,54450,5526,Alcohol,2022-10-06,NaT,AYULA290319921,SAPPHIRE,2023-10-05,Nick.Vanbreda,SAPPHIRE_ITSP_20231005,Ethanol
5,10550,77259,238,Alcohol,2023-06-13,NaT,ENRIM121119722,SAPPHIRE,2023-10-10,Nick.Vanbreda,SAPPHIRE_ITSP_20231010,Ethanol
7,10550,68763,9333,Heroin,2023-03-16,NaT,MIHRE121119802,SAPPHIRE,2023-11-08,Nick.Vanbreda,SAPPHIRE_ITSP_20231108,Heroin
...,...,...,...,...,...,...,...,...,...,...,...,...
105,17751,76562,3367,Methamphetamine,2023-06-06,NaT,LASNG141119902,MURMWIO,2023-11-06,Charles.Debaecker,MURMWIO_INAS_20231106,Methamphetamine
106,17751,38601,8492,"Cannabinoids and Related Drugs, nec",2022-04-20,NaT,OSEEB160219752,MURMPP,2023-10-17,Charles.Debaecker,MURMPP_ITSP_20231017,"Cannabinoids and Related Drugs, n.f.d."
107,17751,38601,8492,"Cannabinoids and Related Drugs, nec",2022-04-20,NaT,OSEEB160219752,MURMPP,2023-11-14,Charles.Debaecker,MURMPP_ITSP_20231114,"Cannabinoids and Related Drugs, n.f.d."
108,17751,69661,9416,Cannabinoids,2023-03-27,NaT,OCEAT020819892,MURMPP,2023-10-04,Mikaela.Feltham,MURMPP_ITSP_20231004,Cannabinoids


In [44]:
atom_df

,SLK,AssessmentDate,Program,Staff,RowKey,PDC
0,ABCDT210719812,2023-10-23,BegaMHHub,Aftab.Jalal,BegaMHHub_INAS_20231023,"Benzodiazepines, n.f.d."
6,ACRAN280320011,2023-11-21,MURMWIO,Craig.Maxwell,MURMWIO_INAS_20231121,"Cannabinoids and Related Drugs, n.f.d."
11,ADIRO150619781,2023-10-23,GOLBICE,Simona.Angeli,GOLBICE_INAS_20231023,"Cannabinoids and Related Drugs, n.f.d."
16,AGIAN010119792,2023-10-05,SAPPHIRE,Molly.Reynolds,SAPPHIRE_ITSP_20231005,Ethanol
21,AHYAT141219641,2023-11-28,SAPPHIRE,Michelle.Wilkie,SAPPHIRE_ITSP_20231128,Ethanol
...,...,...,...,...,...,...
1115,YKSLA240819972,2023-11-17,ALONGSIDE,Lexxie.Jury,ALONGSIDE_ITSP_20231117,Ethanol
1118,YOBLI011019882,2023-10-30,GOLBICE,Anthony.Williams,GOLBICE_INAS_20231030,Methamphetamine
1124,YREAT240819911,2023-11-13,MURMWIO,Michele.Lumb,MURMWIO_INAS_20231113,Methamphetamine
1125,YREMY040119792,2023-10-06,MURMICE,Tina.Raccanello,MURMICE_INAS_20231006,Ethanol
